In [1]:
%env XLA_IR_DEBUG=1
%env XLA_HLO_DEBUG=1

env: XLA_IR_DEBUG=1
env: XLA_HLO_DEBUG=1


In [2]:
import torch
import torch.nn as nn
import torch_xla

In [3]:
torch.set_default_device(torch_xla.device())

In [4]:
torch_xla.sync()
drop = nn.Dropout(p=0.5)
x = torch.tensor([1.0, 2.0, 3.0], dtype=torch.bfloat16)
y = drop(x)
ctx = torch_xla._XLAC.lowering.LoweringContext()
ctx.build([y])
mapping_5 = ctx.parameter_id_tensor_mapping()
hlo_5 = torch_xla._XLAC._get_xla_tensors_hlo([y])
print(hlo_5)

GetRngSeed of device 0x55e4ac9fc260
Init seed value from scalar: 709230394
Update seed to: 151784526842133
HloModule IrToHlo.49, entry_computation_layout={(f32[], s64[], bf16[], bf16[3]{0})->(bf16[3]{0})}

ENTRY %IrToHlo.49 (p0.1: f32[], p1.2: s64[], p2.7: bf16[], p3.44: bf16[3]) -> (bf16[3]) {
  %constant.5 = s64[] constant(2531011), metadata={op_type="prim__Constant" op_name="prim__Constant" source_file="/usr/local/lib/python3.10/site-packages/torch/nn/functional.py" source_line=1425}
  %constant.3 = s64[] constant(214013), metadata={op_type="prim__Constant" op_name="prim__Constant" source_file="/usr/local/lib/python3.10/site-packages/torch/nn/functional.py" source_line=1425}
  %p1.2 = s64[] parameter(1), metadata={op_type="xla__device_data" op_name="xla__device_data" source_file="/usr/local/lib/python3.10/site-packages/torch/nn/functional.py" source_line=1425}
  %multiply.4 = s64[] multiply(s64[] %constant.3, s64[] %p1.2), metadata={op_type="aten__mul" op_name="aten__mul" source_fil

In [5]:
torch_xla.sync()
drop = nn.Dropout(p=0.1)
x = torch.tensor([1.0, 2.0, 3.0], dtype=torch.bfloat16)
y = drop(x)
ctx = torch_xla._XLAC.lowering.LoweringContext()
ctx.build([y])
mapping_1 = ctx.parameter_id_tensor_mapping()
hlo_1 = torch_xla._XLAC._get_xla_tensors_hlo([y])
print(hlo_1)

GetRngSeed of device 0x55e4ac9fc260
Init seed value from scalar: 4973168205254853
Update seed to: 12858234909764943988
HloModule IrToHlo.49, entry_computation_layout={(f32[], s64[], bf16[], bf16[3]{0})->(bf16[3]{0})}

ENTRY %IrToHlo.49 (p0.1: f32[], p1.2: s64[], p2.7: bf16[], p3.44: bf16[3]) -> (bf16[3]) {
  %constant.5 = s64[] constant(2531011), metadata={op_type="prim__Constant" op_name="prim__Constant" source_file="/usr/local/lib/python3.10/site-packages/torch/nn/functional.py" source_line=1425}
  %constant.3 = s64[] constant(214013), metadata={op_type="prim__Constant" op_name="prim__Constant" source_file="/usr/local/lib/python3.10/site-packages/torch/nn/functional.py" source_line=1425}
  %p1.2 = s64[] parameter(1), metadata={op_type="xla__device_data" op_name="xla__device_data" source_file="/usr/local/lib/python3.10/site-packages/torch/nn/functional.py" source_line=1425}
  %multiply.4 = s64[] multiply(s64[] %constant.3, s64[] %p1.2), metadata={op_type="aten__mul" op_name="aten__mul

In [6]:
# Question: why are these HLOs the same?
# Answer: every time we `mark_step`, the `devctx->seed_ir_value` Lazy IR value gets
# reset back to an empty value, essentially resetting the Lazy IR graph representing the
# seed updates. In addition, the `devctx->seed` value gets updated according to a formula
# devctx->seed = 1012031 + devctx->seed * 7012063, so the next time the RNG is invoked,
# we'll get a different `devctx->seed_ir_value`.
hlo_1 == hlo_5

True

In [7]:
import sys
from difflib import unified_diff
sys.stdout.writelines(unified_diff(hlo_1, hlo_5, fromfile="hlo1", tofile="hlo5", n=5))

In [8]:
{ k: (type(v), id(v)) for k, v in mapping_1.items() }

{3: (torch.Tensor, 140268645422064),
 2: (torch.Tensor, 140281431535632),
 1: (torch.Tensor, 140268666958160),
 0: (torch.Tensor, 140268610811232)}

In [9]:
{ k: (type(v), id(v)) for k, v in mapping_5.items() }

{3: (torch.Tensor, 140281580106144),
 2: (torch.Tensor, 140268610983056),
 1: (torch.Tensor, 140268610983376),
 0: (torch.Tensor, 140268610982896)}

In [10]:
mapping_1[3], mapping_5[3]

(tensor([1., 2., 3.], dtype=torch.bfloat16),
 tensor([1., 2., 3.], dtype=torch.bfloat16))

In [11]:
mapping_1[0], mapping_5[0]

(tensor(0.9000), tensor(0.5000))

In [12]:
mapping_1[2], mapping_5[2]

(tensor(0.8984, dtype=torch.bfloat16), tensor(0.5000, dtype=torch.bfloat16))

In [13]:
# This is the random seed.
mapping_1[1], mapping_5[1]

(tensor(4973168205254853), tensor(709230394))

## Test again but don't mark step